### import and set-up

need to replace this code to add your token, i've used mine

In [50]:
from google.colab import userdata
my_token_secret = userdata.get('token_dal')

In [1]:
import os
import json
import pickle
import warnings
from pathlib import Path

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

In [52]:
from huggingface_hub import login
login(token=my_token_secret, add_to_git_credential=False)

file paths (they're on my drive)

In [6]:
TRAIN_PATH = '/content/drive/MyDrive/sem1/kc/train_data.json'
TEST_PATH = '/content/drive/MyDrive/sem1/kc/test_data.json'
METRIC_NAMES_PATH = '/content/drive/MyDrive/sem1/kc/metric_names.json'
METRIC_NAME_EMBD_PATH = '/content/drive/MyDrive/sem1/kc/metric_name_embeddings.npy'

In [2]:
import time
generated_csv = f'/content/drive/MyDrive/sem1/kc/output_{int(time.time())}.csv'

importing those jsons

In [7]:
def _read_json(path):
    with open(path, "r") as handle:
        return json.load(handle)

def _load_array(path):
    return np.load(path)

In [8]:
train_data = _read_json(TRAIN_PATH)
test_data = _read_json(TEST_PATH)
metric_names_list = _read_json(METRIC_NAMES_PATH)

metric_embeddings = _load_array(METRIC_NAME_EMBD_PATH)

In [9]:
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

In [12]:
from collections import Counter

In [13]:
score_stats = Counter(train_df["score"])
print(score_stats)

Counter({'9.0': 3123, '10.0': 1442, '8.0': 259, '7.0': 95, '6.0': 45, '0.0': 13, '3.0': 7, '1.0': 6, '2.0': 5, '4.0': 3, '5.0': 1, '9.5': 1})


In [60]:
for key, value in score_stats.items():
    print(f"{key}: {value}")

10.0: 1442
9.0: 3123
8.0: 259
7.0: 95
6.0: 45
0.0: 13
3.0: 7
2.0: 5
1.0: 6
4.0: 3
5.0: 1
9.5: 1


In [27]:
train_df["score"] = pd.to_numeric(train_df["score"])

### making embd

In [61]:
from sentence_transformers import SentenceTransformer

In [62]:
embedding_model = SentenceTransformer(
    "google/embeddinggemma-300m",
    token=my_token_secret
)

for numeric scores ^^

In [64]:
for col in ["system_prompt", "user_prompt", "response"]:
    train_df[col] = train_df[col].fillna("")
    test_df[col] = test_df[col].fillna("")

filling in the blanks, there were some

now making the actual embeddings

In [65]:
import time

In [66]:
start = time.time()
train_user_emb   = embedding_model.encode(
    train_df["user_prompt"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
)
end = time.time()
time_taken_1 = end - start
print(f'{(time_taken_1):.2f}')

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

49.75


In [67]:
start = time.time()
train_sys_emb    = embedding_model.encode(
    train_df["system_prompt"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
)
end = time.time()
time_taken_2 = end - start
print(f'{(time_taken_2):.2f}')

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

14.78


In [68]:
start = time.time()
train_resp_emb   = embedding_model.encode(
    train_df["response"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
)
end = time.time()
time_taken_3 = end - start
print(f'{(time_taken_3):.2f}')

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

175.06


In [69]:
start = time.time()
test_user_emb    = embedding_model.encode(
    test_df["user_prompt"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
)
end = time.time()
time_taken_4 = end - start
print(f'{(time_taken_4):.2f}')

Batches:   0%|          | 0/114 [00:00<?, ?it/s]

40.84


In [70]:
start = time.time()
test_sys_emb     = embedding_model.encode(
    test_df["system_prompt"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
)
end = time.time()
time_taken_5 = end - start
print(f'{(time_taken_5):.2f}')

Batches:   0%|          | 0/114 [00:00<?, ?it/s]

11.12


In [71]:
start = time.time()
test_resp_emb    = embedding_model.encode(
    test_df["response"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
)
end = time.time()
time_taken_6 = end - start
print(f'{(time_taken_6):.2f}')

Batches:   0%|          | 0/114 [00:00<?, ?it/s]

127.16


In [72]:
total_time_for_embeddings = time_taken_1 + time_taken_2 + time_taken_3 + time_taken_4 + time_taken_5 + time_taken_6
print(f'{(total_time_for_embeddings//60):.0f} mins, {(total_time_for_embeddings%60):.2f} secs')

6 mins, 58.70 secs


saving these to my drive

In [73]:
output_dir = '/content/drive/MyDrive/sem1/kc/'

In [74]:
np.save(output_dir + 'train_user_emb.npy',   train_user_emb)
np.save(output_dir + 'train_sys_emb.npy',    train_sys_emb)
np.save(output_dir + 'train_resp_emb.npy',   train_resp_emb)

np.save(output_dir + 'test_user_emb.npy',    test_user_emb)
np.save(output_dir + 'test_sys_emb.npy',     test_sys_emb)
np.save(output_dir + 'test_resp_emb.npy',    test_resp_emb)

### wting

In [14]:
score_wts = len(train_df) / (len(score_stats) * train_df["score"].value_counts())

### making a cosine similarity matrix

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
output_dir = "/content/drive/MyDrive/sem1/kc/"

In [17]:
train_user_emb  = np.load(output_dir + "train_user_emb.npy")
train_sys_emb   = np.load(output_dir + "train_sys_emb.npy")
train_resp_emb  = np.load(output_dir + "train_resp_emb.npy")

test_user_emb   = np.load(output_dir + "test_user_emb.npy")
test_sys_emb    = np.load(output_dir + "test_sys_emb.npy")
test_resp_emb   = np.load(output_dir + "test_resp_emb.npy")

In [18]:
metric_embeddings_norm = metric_embeddings / np.linalg.norm(
    metric_embeddings, axis=1, keepdims=True
)

In [19]:
metric_to_idx = {name: idx for idx, name in enumerate(metric_names_list)}

In [20]:
metric_sim_mat = cosine_similarity(metric_embeddings_norm)
np.fill_diagonal(metric_sim_mat, 0)

In [21]:
print(metric_sim_mat.shape)
print(f"[{metric_sim_mat.min():.3f}, {metric_sim_mat.max():.3f}]")

(145, 145)
[0.000, 0.984]


for all metrics, i've made similarity matrix by calculating pairwise similarity (cosine)

this is to get an idea of which metrics are closer in meaning

In [22]:
print(metric_sim_mat)

[[0.         0.9169104  0.945053   ... 0.70465803 0.67319894 0.68820393]
 [0.9169104  0.         0.9182794  ... 0.69642097 0.65991354 0.69468766]
 [0.945053   0.9182794  0.         ... 0.7135308  0.6729646  0.6925874 ]
 ...
 [0.70465803 0.69642097 0.7135308  ... 0.         0.74057853 0.79578596]
 [0.67319894 0.65991354 0.6729646  ... 0.74057853 0.         0.7743733 ]
 [0.68820393 0.69468766 0.6925874  ... 0.79578596 0.7743733  0.        ]]


comparing with metric names in the data, this seems to go well

### binning metrics based on how close they are to each other

In [23]:
metric_buckets = {}

In [24]:
for m_idx, m_name in enumerate(metric_names_list):
    sims = metric_sim_mat[m_idx]

    cut20 = np.percentile(sims, 20)
    cut40 = np.percentile(sims, 40)
    cut60 = np.percentile(sims, 60)
    cut80 = np.percentile(sims, 80)

    metric_buckets[m_name] = {
        "far_off": [
            metric_names_list[i] for i, s in enumerate(sims)
            if s <= cut20 and i != m_idx
        ],
        "kinda_far": [
            metric_names_list[i] for i, s in enumerate(sims)
            if cut20 < s <= cut40 and i != m_idx
        ],
        "mid_zone": [
            metric_names_list[i] for i, s in enumerate(sims)
            if cut40 < s <= cut60 and i != m_idx
        ],
        "kinda_close": [
            metric_names_list[i] for i, s in enumerate(sims)
            if cut60 < s <= cut80 and i != m_idx
        ],
        "super_close": [
            metric_names_list[i] for i, s in enumerate(sims)
            if s > cut80 and i != m_idx
        ],
    }

In [25]:
def make_some_extra_samples(df, u_emb, s_emb, r_emb, chunk=12):
    """
    Make extra rows using the metric similarity buckets.
    Rows with score >= 7 act as anchors for generating variations.
    """

    add_on = []

    anchors = df[df["score"] >= 7].copy()
    print(f"Cooking up {len(anchors)} anchor rows...")

    for idx, row in anchors.iterrows():
        base = row["metric_name"]
        grp = metric_buckets[base]

        # 1) far_off → score 0–2
        for _ in range(int(chunk * 0.20)):
            if grp["far_off"]:
                m = np.random.choice(grp["far_off"])
                sc = np.random.choice([0, 1, 2], p=[0.5, 0.3, 0.2])

                add_on.append({
                    "metric_name": m,
                    "user_prompt": row["user_prompt"],
                    "system_prompt": row["system_prompt"],
                    "response": row["response"],
                    "score": sc,
                    "kind": "far_off",
                    "root": idx,
                    "root_score": row["score"],
                })

        # 2) kinda_far → score 2–4
        for _ in range(int(chunk * 0.25)):
            if grp["kinda_far"]:
                m = np.random.choice(grp["kinda_far"])
                sc = np.random.choice([2, 3, 4], p=[0.3, 0.4, 0.3])

                add_on.append({
                    "metric_name": m,
                    "user_prompt": row["user_prompt"],
                    "system_prompt": row["system_prompt"],
                    "response": row["response"],
                    "score": sc,
                    "kind": "kinda_far",
                    "root": idx,
                    "root_score": row["score"],
                })

        # 3) mid_zone → score 4–6
        for _ in range(int(chunk * 0.30)):
            if grp["mid_zone"]:
                m = np.random.choice(grp["mid_zone"])
                sc = np.random.choice([4, 5, 6], p=[0.3, 0.4, 0.3])

                add_on.append({
                    "metric_name": m,
                    "user_prompt": row["user_prompt"],
                    "system_prompt": row["system_prompt"],
                    "response": row["response"],
                    "score": sc,
                    "kind": "mid_zone",
                    "root": idx,
                    "root_score": row["score"],
                })

        # 4) kinda_close → score 5–7
        for _ in range(int(chunk * 0.25)):
            if grp["kinda_close"]:
                m = np.random.choice(grp["kinda_close"])
                sc = np.random.choice([5, 6, 7], p=[0.3, 0.4, 0.3])

                add_on.append({
                    "metric_name": m,
                    "user_prompt": row["user_prompt"],
                    "system_prompt": row["system_prompt"],
                    "response": row["response"],
                    "score": sc,
                    "kind": "kinda_close",
                    "root": idx,
                    "root_score": row["score"],
                })

    extra_df = pd.DataFrame(add_on)

    # keep originals marked too
    orig_df = df.assign(
        kind="orig",
        root=df.index,
        root_score=df["score"],
    )

    final = pd.concat([orig_df, extra_df], ignore_index=True)
    return final

In [28]:
more_rows_df = make_some_extra_samples(
    train_df,
    train_user_emb,
    train_sys_emb,
    train_resp_emb,
    chunk=12
)

Cooking up 4920 anchor rows...


In [29]:
for k in ["far_off", "kinda_far", "mid_zone", "kinda_close"]:
    cnt = (more_rows_df["kind"] == k).sum()
    print(f"  {k}: {cnt}")

  far_off: 9840
  kinda_far: 14760
  mid_zone: 14760
  kinda_close: 14760


### emd for new cooked up daeta

In [30]:
all_user_embs = []
all_sys_embs  = []
all_resp_embs = []

In [31]:
for _, row in more_rows_df.iterrows():
    root_i = row["root"]

    all_user_embs.append(train_user_emb[root_i])
    all_sys_embs.append(train_sys_emb[root_i])
    all_resp_embs.append(train_resp_emb[root_i])

In [32]:
all_user_embs = np.array(all_user_embs)
all_sys_embs  = np.array(all_sys_embs)
all_resp_embs = np.array(all_resp_embs)

metric embeddings for all the rows

appending from the metric embeddings df

In [33]:
all_metric_embs = []

for _, row in more_rows_df.iterrows():
    mname = row["metric_name"]

    if mname in metric_to_idx:
        all_metric_embs.append(metric_embeddings_norm[metric_to_idx[mname]])
    else:
        all_metric_embs.append(metric_embeddings_norm.mean(axis=0))

all_metric_embs = np.array(all_metric_embs)

### making new features

In [34]:
def mix_features(m_emb, u_emb, s_emb, r_emb):
    '''
    metric + user+ sys +resp
    '''

    # dotting
    sims = np.array([
        (m_emb * u_emb).sum(axis=1),
        (m_emb * s_emb).sum(axis=1),
        (m_emb * r_emb).sum(axis=1),
        (u_emb * r_emb).sum(axis=1),
        (u_emb * s_emb).sum(axis=1),
        (s_emb * r_emb).sum(axis=1),

        # combos
        (m_emb * (u_emb + r_emb) / 2).sum(axis=1),
        (m_emb * (u_emb + s_emb + r_emb) / 3).sum(axis=1),
    ]).T

    # summary stats
    sim_stats = np.column_stack([
        sims.mean(axis=1),
        sims.std(axis=1),
        sims.max(axis=1),
        sims.min(axis=1),
    ])

    # another small blend
    mid_mix = m_emb * (u_emb + r_emb) / 2

    # everything together
    feats = np.concatenate([
        m_emb,
        u_emb,
        s_emb,
        r_emb,
        sims,
        sim_stats,
        mid_mix,
    ], axis=1)

    return feats

for features, i've mixed the mentioned ones along with pairwise dots, and other compositie mixes



In [35]:
X_train_more = mix_features(
    all_metric_embs,
    all_user_embs,
    all_sys_embs,
    all_resp_embs
)

for test metric

In [36]:
test_m_embs = []

In [37]:
for _, row in test_df.iterrows():
    mname = row["metric_name"]

    if mname in metric_to_idx:
        test_m_embs.append(metric_embeddings_norm[metric_to_idx[mname]])
    else:
        # fallback if metric name isn't known
        test_m_embs.append(metric_embeddings_norm.mean(axis=0))

In [38]:
test_m_embs = np.array(test_m_embs)

In [39]:
X_test = mix_features(
    test_m_embs,
    test_user_emb,
    test_sys_emb,
    test_resp_emb
)

### training ensemble models

In [40]:
row_wts = np.ones(len(more_rows_df))

originals are the ones with kind == 'orig'

In [42]:
orig_mask = more_rows_df["kind"] == "orig"
orig_scores = more_rows_df.loc[orig_mask, "score"]
score_counts = orig_scores.value_counts()
max_count = score_counts.max()

giving rare scores more weight

In [43]:
for idx, row in more_rows_df[orig_mask].iterrows():
    row_wts[idx] = max_count / score_counts[row["score"]]

giving the extra/generated rows a small boost

In [44]:
row_wts[~orig_mask] = 2.0

blanks for predictions

In [45]:
train_preds = np.zeros(len(train_df))
test_preds  = np.zeros(len(test_df))

k folding

In [46]:
from sklearn.model_selection import GroupKFold

In [47]:
folds = GroupKFold(n_splits=4)

In [48]:
groups = []
for idx, row in more_rows_df.iterrows():
    if row["kind"] == "orig":
        groups.append(row["metric_name"])
    else:
        groups.append(f"extra_{row['metric_name']}_{idx}")

groups = np.array(groups)

### xgb model

In [49]:
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [50]:
for fold, (tr_idx, val_idx) in enumerate(folds.split(X_train_more,
                                                     more_rows_df["score"],
                                                     groups), 1):

    print(f"\n--- starting fold #{fold} ---")

    # validate only on original rows
    val_orig_mask = (more_rows_df.iloc[val_idx]["kind"] == "orig").values
    orig_val_ids  = more_rows_df.iloc[val_idx][val_orig_mask]["root"].values

    dtrain = xgb.DMatrix(
        X_train_more[tr_idx],
        label=more_rows_df["score"].values[tr_idx],
        weight=row_wts[tr_idx]
    )

    dval = xgb.DMatrix(
        X_train_more[val_idx][val_orig_mask],
        label=more_rows_df["score"].values[val_idx][val_orig_mask]
    )

    dtest = xgb.DMatrix(X_test)

    params = {
        "objective": "reg:squarederror",
        "max_depth": 6,
        "learning_rate": 0.03,
        "subsample": 0.85,
        "colsample_bytree": 0.85,
        "reg_alpha": 0.1,
        "reg_lambda": 1.0,
        "random_state": 42,
        "tree_method": "hist",
    }

    model = xgb.train(
        params,
        dtrain,
        num_boost_round=600,
        evals=[(dval, "val")],
        early_stopping_rounds=40,
        verbose_eval=False
    )

    # store preds
    val_preds = model.predict(dval)
    train_preds[orig_val_ids] = val_preds
    test_preds += model.predict(dtest) / 4   # 4 folds

    # evaluate
    rmse = np.sqrt(mean_squared_error(
        train_df.loc[orig_val_ids, "score"],
        np.clip(np.round(val_preds), 0, 10)
    ))

    print(f"done with fold #{fold}, got rmse {rmse:.4f}")

    # nicer distribution readout
    vals = pd.Series(np.clip(np.round(val_preds), 0, 10)).value_counts().sort_index()

    low  = vals[vals.index <= 4].sum() if any(vals.index <= 4) else 0
    mid  = vals[(vals.index >= 5) & (vals.index <= 7)].sum() if any((vals.index >= 5) & (vals.index <= 7)) else 0
    high = vals[vals.index >= 8].sum() if any(vals.index >= 8) else 0

    print(f"fold {fold} preds → low:{low}, mid:{mid}, high:{high}")


--- starting fold #1 ---
done with fold #1, got rmse 3.7806
fold 1 preds → low:221, mid:988, high:41

--- starting fold #2 ---
done with fold #2, got rmse 4.2428
fold 2 preds → low:351, mid:842, high:58

--- starting fold #3 ---
done with fold #3, got rmse 3.7302
fold 3 preds → low:228, mid:963, high:59

--- starting fold #4 ---
done with fold #4, got rmse 4.1108
fold 4 preds → low:357, mid:824, high:68


In [51]:
final_predictions = np.clip(np.round(test_preds), 0, 10)

In [52]:
submission_file = pd.DataFrame({
    "ID": range(1, len(final_predictions) + 1),
    "score": final_predictions
})

In [53]:
submission_file.to_csv(generated_csv, index=False)
print(f"saved submission to: {generated_csv}")

saved submission to: /content/drive/MyDrive/sem1/kc/output_1763528559.csv


In [54]:
print("how the test scores are distributed:")

total = len(submission_file)

for sc in sorted(submission_file["score"].unique()):
    count = (submission_file["score"] == sc).sum()
    pct   = 100 * count / total
    dots  = "•" * int(pct / 4)

    print(f"   {int(sc):2d}   | {count:6d} |  {pct:6.2f}% | {dots}")

how the test scores are distributed:
    0   |      1 |    0.03% | 
    1   |     18 |    0.49% | 
    2   |     98 |    2.69% | 
    3   |    253 |    6.95% | •
    4   |    465 |   12.78% | •••
    5   |    794 |   21.83% | •••••
    6   |    852 |   23.42% | •••••
    7   |    645 |   17.73% | ••••
    8   |    417 |   11.46% | ••
    9   |     91 |    2.50% | 
   10   |      4 |    0.11% | 
